### Library and data needed

In [50]:
import os
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [51]:
opts_k = 10
opts_rho = 0.1
opts_replicates = 100
opts_maxiter = 1000
np.random.seed(0)


### Preprocessing : Read file and prepare input 

In [52]:
# # reading file
os.chdir('../backend')
x = pd.read_csv(os.getcwd() + "/x.csv", header=None)
y = pd.read_csv(os.getcwd() + "/y.csv", header=None)

In [53]:
# Skip feature selection if <= 3 features and all features are kept
if x.shape[1] <= 3:
    print("There are 3 or less features to do selection. Skipping feature selection.")
    # TODO: return the desire output format
elif x.shape[1] <= 1:
    print("There is only 1 feature. Stopping space construction.")
    # TODO: return the desire output format
    
n_feature = x.shape[1]

### Features Selection (Correlation Based): Line 39-53)


In [54]:
# Initialize empty matrices for correlation coefficients and p-values
corr_matrix = pd.DataFrame(index=x.columns, columns=y.columns)
p_value_matrix = pd.DataFrame(index=x.columns, columns=y.columns)

# Compute correlation coefficient and p-value for each pair of variables
for x_col in x.columns:
    for y_col in y.columns:
        corr_coef, p_value = pearsonr(x[x_col], y[y_col])
        # assign correlation value to 0 so that it can be filtered later
        if p_value > 0.05:
            corr_matrix.loc[x_col, y_col] = 0    
        else:
            corr_matrix.loc[x_col, y_col] = corr_coef
corr_matrix = abs(corr_matrix)

# TODO: testing: check corr_matrix (manual check pass)

In [55]:
corr_matrix

,0,1,2,3,4,5,6,7,8,9
0,0,0.16947,0,0.15147,0,0,0,0,0,0
1,0.183991,0.30404,0.603926,0.162437,0,0,0,0.157502,0,0.188585
2,0.515083,0.216131,0,0.328626,0.370241,0.511744,0.427149,0.36599,0.460169,0.296477
3,0.508291,0.464738,0.357001,0.564691,0.526048,0.496176,0.522701,0.682551,0.537335,0.390996
4,0.186968,0.288352,0.233814,0.379854,0.352313,0.324384,0.307727,0.427858,0.371381,0.259739
5,0.538544,0.162534,0,0.507152,0.54086,0.519021,0.485531,0.462534,0.468816,0.346111
6,0.327107,0.158215,0,0.376153,0.390311,0.408971,0.40457,0.369844,0.354679,0.280685
7,0.623523,0.444072,0,0.658937,0.658274,0.667203,0.745088,0.709117,0.727195,0.502527
8,0.713447,0.475969,0.141597,0.812574,0.82579,0.88434,0.795783,0.784334,0.825643,0.643079
9,0.496212,0.184281,0,0.354537,0.385519,0.39823,0.466231,0.307199,0.417937,0.330138


In [56]:
# choose features with correlatiton value over 0.01 (specified in options.json)
# filter out x values with selected feature
for column in corr_matrix.columns:
    corr_matrix[column] = corr_matrix[column].apply(lambda x: 0 if x < opts_rho else x)

# choose features (only choose feautres that all correlation value >= opts_rho)
features_selected = []
for index, row in corr_matrix.iterrows():
    if (row != 0).any():
        features_selected.append(index)
        
x_aux = x.iloc[:,features_selected]
# TODO: test, compare xaux 
x_aux.shape

(212, 10)

### K-means Clustering (Line 69-95)

- Applies k-means clustering algorithm to Xaux
- Evaluates sihouette values for different number of clusters
- Output -> (1) inspected num of clusters, (2) corresponding criterion values
- Checks, if sihouette value for specified number of clusters is below 0.5 (sihouette value is  a measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation), the range is -1 to 1)
    - we do average sihouette value for each cluster
    - if ASV is < 0.5 ==> poor cluster quality and suggest increasing opts.K. Or it will find a value of K that yields sihouette value above 7.5, if available
- After checking sihouette value, do k-mean clustering

***Steps***
1. construct kmc and compute silhouette_score for chosen k clusters
2. repeat step 1 for k clusters value ranging from 3 to n_feature
3. display average of silhouette score based on k clusters
3. report if your chosen k-mean based on options.json is lower than 0.5
    - if lower than 0.5, based on step 2, find the best k clusters that has silhouette score higher than 7.5

***k clusters***
- using k value from options.json, do kmc

In [57]:
# Initialize empty dictionary to store silhouette scores
silhouette_scores = {}

# Range of cluster numbers to evaluate
min_clusters = 3
max_clusters = n_feature  # Assuming nfeats is defined
cluster_range = range(min_clusters, max_clusters + 1)

# # Evaluating silhouette scores for different numbers of clusters
for n_clusters in cluster_range:
    kmeans = KMeans(n_clusters=n_clusters, n_init = n_clusters, random_state=0)
    cluster_labels = kmeans.fit_predict(x_aux)  # Transpose Xaux for clustering
    silhouette_scores[n_clusters] = silhouette_score(x_aux, cluster_labels, metric='correlation')

# Displaying average silhouette values for each number of clusters
print('-> Average silhouette values for each number of clusters.')
for n_clusters, silhouette_score in silhouette_scores.items():
    print(f"Number of clusters: {n_clusters}, Silhouette score: {silhouette_score}")

# Checking silhouette value for opts.K
if silhouette_scores[opts_k] < 0.5:
    print(f'-> The silhouette value for K={opts_k} is below 0.5. You should consider increasing K.')
    K_suggested = next((k for k, score in silhouette_scores.items() if score > 0.75), None)
    if K_suggested is not None:
        print(f'A suggested value of K is {K_suggested}.')
    else:
        print(f'No suggested value of K')
# TODO: silhouette score is way different, perhaps the x_aux is very different

-> Average silhouette values for each number of clusters.
Number of clusters: 3, Silhouette score: 0.4059232471018759
Number of clusters: 4, Silhouette score: 0.34508112245767003
Number of clusters: 5, Silhouette score: 0.3543138996675518
Number of clusters: 6, Silhouette score: 0.3576262008062844
Number of clusters: 7, Silhouette score: 0.30190541309710284
Number of clusters: 8, Silhouette score: 0.2747242666978954
Number of clusters: 9, Silhouette score: 0.2682247181262402
Number of clusters: 10, Silhouette score: 0.2715311894295123
-> The silhouette value for K=10 is below 0.5. You should consider increasing K.
No suggested value of K


In [58]:
x_aux.T.shape

(10, 212)

In [69]:
# Do k means with k value defined in options.json
kmeans = KMeans(n_clusters=opts_k, 
                init='k-means++',
                n_init=opts_replicates,
                max_iter=opts_maxiter,
                random_state=0)
predict = kmeans.fit_predict(x_aux.T)  # Transpose Xaux for clustering
predict
# labels = kmeans.labels_
# out_clust = np.zeros((opts_k, x_aux.T.shape[0]), dtype=bool)
# out_clust.shape

# # Populate out.clust based on the cluster labels
# for i in range(opts_k):
#     out_clust[i, :] = labels == i

# # Transpose out.clust to match MATLAB's behavior
# c = out_clust.T

array([4, 6, 3, 7, 2, 0, 8, 1, 9, 5], dtype=int32)

## Choosing Combinations 

- [X1,X2,X3,X4,X5,X6,X7,X8,X9,X10] = ndgrid(find(out.clust(:,1)),find(out.clust(:,2)),find(out.clust(:,3)),find(out.clust(:,4)),find(out.clust(:,5)),find(out.clust(:,6)),find(out.clust(:,7)),find(out.clust(:,8)),find(out.clust(:,9)),find(out.clust(:,10))); = 

- comb = [X1(:),X2(:),X3(:),X4(:),X5(:),X6(:),X7(:),X8(:),X9(:),X10(:)]; = 

- ncomb = size(comb,1); %#ok<*NODEF>
- comb = sort(comb,2);

In [ ]:
# Initialize variables
X = []
find_commands = []

# Generate X variables and find commands
for i in range(1, opts.K + 1):
    X_var = f'X{i}'
    find_command = f'find(out.clust[:, {i}])'
    X.append(X_var)
    find_commands.append(find_command)

# Execute find commands and assign to X variables
X_vars = np.meshgrid(*[eval(command) for command in find_commands])

# Combine X variables
comb = np.column_stack([X_var.ravel() for X_var in X_vars])

# Calculate number of combinations
ncomb = comb.shape[0]

# Sort combinations
comb.sort(axis=1)

print(f'-> {ncomb} valid feature combinations.')
